In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [4]:
# Load the data
df = pd.read_csv('data\\02_stg\\stg_disaster_messages.csv')
df.head(n=2)

C:\Users\grant\AppData\Local\Temp\ipykernel_23388\3261564891.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data\\02_stg\\stg_disaster_messages.csv')


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


In [14]:
#Filter df where genre is 'direct', related is 1 and request is 1
df_filtered = df[(df['genre'] == 'direct') & (df['related'] == 1) & (df['request'] == 1)]

# Drop all columns except 'message', 'request', 'offer', 'direct_report'
df_filtered = df_filtered[['message', 'request', 'offer', 'direct_report']]

#Create a new column called 'message_type' which returns 'request' if request is 1, 'offer' if offer is 1 and 'direct_report' if direct_report is 1
df_filtered['message_type'] = df_filtered.apply(
    lambda x: 'request' if x['request'] == 1 else ('offer' if x['offer'] == 1 else ('direct_report' if x['direct_report'] == 1 else 'unknown')), 
    axis=1
)

# Drop the 'request', 'offer' and 'direct_report' columns
df_filtered = df_filtered.drop(columns=['request', 'offer', 'direct_report'])

#drop duplicates
df_filtered = df_filtered.drop_duplicates()

df_filtered.head()

,message,message_type
3,UN reports Leogane 80-90 destroyed. Only Hospi...,request
7,"Please, we need tents and water. We are in Sil...",request
9,I am in Croix-des-Bouquets. We have health iss...,request
10,"There's nothing to eat and water, we starving ...",request
12,"I am in Thomassin number 32, in the area named...",request


In [15]:
#check for duplicates
df_filtered.duplicated().sum()

0

In [23]:
import warnings

def classify_messages_types(df):
    # Filter df where genre is 'direct', related is 1 and request is 1
    df_filtered = df[(df['genre'] == 'direct') & (df['related'] == 1)]

    # Drop all columns except 'message', 'request', 'offer', 'direct_report'
    df_filtered = df_filtered[['message', 'request', 'offer', 'direct_report']]

    # Create a new column called 'message_type' which returns 'request' if request is 1, 'offer' if offer is 1 and 'direct_report' if direct_report is 1
    df_filtered['message_type'] = df_filtered.apply(
        lambda x: 'offer' if x['offer'] == 1 else ('request' if x['request'] == 1 else ('direct_report' if x['direct_report'] == 1 else 'other')),
        axis=1
    )

    # Drop the 'request', 'offer' and 'direct_report' columns
    df_filtered = df_filtered.drop(columns=['request', 'offer', 'direct_report'])

    # Check if the number of duplicates is greater than 1% of the DataFrame's length
    num_duplicates = df_filtered.duplicated().sum()
    if num_duplicates > len(df_filtered) * 0.01:
        warnings.warn(f"Dropping {num_duplicates} duplicates, which is more than 1% of the DataFrame's length")

    # Drop duplicates
    df_filtered = df_filtered.drop_duplicates()

    return df_filtered

In [25]:
import plotly.graph_objs as go

def create_plotly_graph(df):
    # Count the number of messages per message_type
    message_types_count = df['message_type'].value_counts()

    # Create a dictionary representing a Plotly graph object
    graph = {
        'data': [
            go.Bar(
                x=message_types_count.index.tolist(),
                y=message_types_count.values.tolist(),
                name='Count'
            )
        ],

        'layout': {
            'title': 'Number of Messages per Message Type',
            'xaxis': {
                'title': "Message Type"
            },
            'yaxis': {
                'title': "Number of Messages"
            },
            'barmode': 'stack'
        }
    }

    return graph

In [26]:
messsage_types_df = classify_messages_types(df)

message_type_graph = create_plotly_graph(messsage_types_df)
print(message_type_graph)

{'data': [Bar({
    'name': 'Count', 'x': ['request', 'unknown', 'direct_report', 'offer'], 'y': [3689, 2935, 504, 46]
})], 'layout': {'title': 'Number of Messages per Message Type', 'xaxis': {'title': 'Message Type'}, 'yaxis': {'title': 'Number of Messages'}, 'barmode': 'stack'}}
